In [242]:
''' Import statements'''
%matplotlib inline
import io
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from scipy.sparse import csr_matrix
from numpy.linalg import inv
from scipy.spatial.distance import pdist, squareform

In [243]:
''' Function to read the data and categorize them into X and Y
    returns :
             x - list of feature vectors
             y - list of given predicted values
             xr - nd-array of feature vectors
             yr - nd-array of given predicted values

'''
def read_data(data_file): 
    
    f = open(data_file,'r')
    x=[]
    y=[]
    yr=[]
    for line in f:
        if not line.startswith('#'):
            sp=line.split()
            #x1=float(sp[0])
            #x2=float(sp[1])
            #x.append((x1,x2))
            y.append(float(sp[-1]))
            x.append([float(sp[i]) for i in range(len(sp)-1)])
            b= [tuple(i) for i in x]
        
    return x,y
    
     

In [244]:
x,y=read_data("mvar-set1.dat")
print x[:10]

[[1.673469387755102, 0.4489795918367345], [-0.0408163265306124, 0.530612244897959], [-0.7755102040816327, -1.591836734693878], [-2.0, -1.346938775510204], [0.7755102040816324, -0.4489795918367348], [-1.26530612244898, 1.510204081632653], [-1.836734693877551, -0.6122448979591838], [-1.428571428571429, 1.755102040816326], [-0.5306122448979593, 2.0], [1.591836734693877, -0.0408163265306124]]


In [245]:
''' Function to return the desired classifier'''

def get_clf():
    return linear_model.LinearRegression()

In [246]:
'''Function to return the transformed matrix
   Input 1 : Feature vector 
   Input 2: Desired degree
   Output : 'Z' - the tranformed matrix eg., [1 x x**2 ....]

'''

def Z_Transform(x,degree):
        poly = PolynomialFeatures(degree)
        z=poly.fit_transform(np.array(x))
        return z


In [247]:
'''Function to calculate the theta values
   Input 1: 'Z' - the tranformed matrix eg., [1 x x**2 ....]
   Input 2: Y values
   Output: theta according to the order eg if degree = 2 , 3 theta values will be returned
'''

def poly_theta(z,y):
    theta = np.dot(np.linalg.pinv(z),y)
    return theta

In [248]:
z=Z_Transform(x,3)
thetas=poly_theta(z,y)
print 'Solution to the regression problem using iterative solution:'
print thetas

Solution to the regression problem using iterative solution:
[  1.02230218e+00   1.00248539e+00   1.00537442e+00  -1.26097513e-02
  -8.47393334e-03  -6.43213699e-03  -2.31159943e-03  -1.64092663e-02
   5.84897669e-04   3.15715316e-03]


In [249]:
'''
Function used in gradient descent to predict values in each iteration
'''

def predict_grad(theta,z):
        return np.dot(theta,z)


In [250]:
'''Function to predict the output of the given z
   Input 1: Thetas
   Input 2: Input 1: 'Z' - the tranformed matrix eg., [1 x x**2 ....]
   Output: Predictions of the given Z matrix
 
'''
def predict(theta,z):
    predictions = []
    for i in z:
        predictions.append(np.dot(theta.transpose(),np.array(i)))
    return predictions

In [251]:
'''Function to calculate the mse 
   Input 1: Predicted values
   Input 2: True values
   Output: mse of predicted and truth 
'''


def mse_calculator(pred,y):
    return sum([(i-j)**2 for i,j in zip(pred,y)])/len(y) # formula to calulate mse - 1/m*sum(yhat-y)**2
    

In [252]:
'''Function to calculate the rse 
   Input 1: Predicted values
   Input 2: True values
   Output: rse of predicted and truth 
'''

def rse_calculator(pred,y):
    return sum([(i-j)**2/y**2 for i,j in zip(pred,y)])/len(y) # formula to calulate mse - 1/m*sum(yhat-y)**2/y**2
    

In [253]:
'''Function to perform 10-fold cross validation. In this method the test and train indices are split using using the inbuit
   'KFold' function.
    Input 1: 'Z' - the tranformed matrix eg., [1 x x**2 ....]
    Input 2: True predicted values
    Input 3: No of folds (10 by default)
    
    Performance measures such as mean suared error and relative suared error are calculated in this function

'''


def poly_ten_fold(x,y,nfolds=10):
    cv = KFold(len(y), nfolds) #inbuilt function to split the indices
    MSE_tr=[]
    MSE_ts=[]
    RSE_tr=[]
    RSE_ts=[]
    for train_idx, test_idx in cv:
        y_=np.array(y)
        theta = poly_theta(x[train_idx],y_[train_idx]) # Find theta for the train set
        pred_train=predict(theta,x[train_idx]) # Predict values using the formula theta.transpose * z for train set
        error_tr = mse_calculator(pred_train,y_[train_idx]) #calculate mse for train
        MSE_tr.append(error_tr)
        error_rse_tr = rse_calculator(pred_train,y_[train_idx]) #calculate rse for train 
        RSE_tr.append(error_rse_tr)
        pred_test = predict(theta,x[test_idx])  # Predict values using the formula theta.transpose * z for test set
        error_ts = mse_calculator(pred_test,y_[test_idx]) #calculate mse for test 
        error_rse_ts = rse_calculator(pred_test,y_[test_idx]) #calculate rse for test 
        MSE_ts.append(error_ts)
        RSE_ts.append(error_rse_ts)
    print 'MSE of train: %f' %np.mean(MSE_tr)
    print 'MSE of test: %f' %np.mean(MSE_ts)

In [254]:
Z=Z_Transform(x,2)
poly_ten_fold(Z,y)

MSE of train: 0.258185
MSE of test: 0.259617


In [255]:
'''Function to perform 10-fold cross validation. In this method the test and train indices are split programatically without the 
   use of in-built function  
    Input 1: 'Z' - the tranformed matrix eg., [1 x x**2 ....]
    Input 2: True predicted values
    Input 3: No of folds (10 by default)
    
    Performance measures such as mean suared error and relative suared error are calculated in this function

'''

def split_indices_cross_fold(x,y,nfolds=10):
    
    split = int((len(y))/nfolds) #split size of the data
    idx=[i for i in range(len(y))] # indices list [0 .... size(data)-1]
    i=0
    j=1
    MSE_tr=[]
    MSE_ts=[]
    RSE_tr=[]
    RSE_ts=[]
    while j <  nfolds+1: # repeat n-fold times
        test_idx=idx[split*i:split*j] # Get test indices
        train_idx=list(set(idx)-set(test_idx)) # Get test indices
        y_=np.array(y)
        theta = poly_theta(x[train_idx],y_[train_idx])
        pred_train=predict(theta,x[train_idx])
        error_tr = mse_calculator(pred_train,y_[train_idx])
        MSE_tr.append(error_tr)
        pred_test = predict(theta,x[test_idx])
        error_ts = mse_calculator(pred_test,y_[test_idx])
        MSE_ts.append(error_ts)
        error_rse_tr = rse_calculator(pred_train,y_[train_idx]) #calculate rse for train 
        RSE_tr.append(error_rse_tr)
        error_rse_ts = rse_calculator(pred_test,y_[test_idx]) #calculate rse for test
        RSE_tr.append(error_rse_ts)
        i+=1
        j+=1
    print 'MSE of train: %f' %np.mean(MSE_tr)
    print 'MSE of test: %f' %np.mean(MSE_ts)
    
    
     

In [256]:
for i in range(2,7):
    Z=Z_Transform(x,i)
    split_indices_cross_fold(Z,y)

MSE of train: 0.258185
MSE of test: 0.259617
MSE of train: 0.257498
MSE of test: 0.260660
MSE of train: 0.256295
MSE of test: 0.260305
MSE of train: 0.255962
MSE of test: 0.261528
MSE of train: 0.254778
MSE of test: 0.261960


In [257]:
'''Function to perform 10-fold cross validation using ready made python functions. This function is used to compare the results
   of the earlier obtained model
    Input 1: 'Z' - the tranformed matrix eg., [1 x x**2 ....]
    Input 2: True predicted values
    Input 3: No of folds (10 by default)
    
    Performance measures such as mean suared error is calculated in this function

'''



def poly_ten_fold_validation_verify(x,y,nfolds=10):
    cv = KFold(len(y), nfolds)
    MSE_tr=[]
    MSE_ts=[]
    RSE_tr=[]
    RSE_ts=[]
    error_tr = 0
    error_ts = 0
    for train_idx, test_idx in cv:
        t = csr_matrix(x).toarray() # convert the train data into CSR matrix
        y_=np.array(y)
        clf = get_clf() # get the classifier
        clf.fit(t[train_idx], y_[train_idx]) # fit the data
        pred_test = clf.predict(t[test_idx]) 
        pred_train = clf.predict(t[train_idx])
        error_ts += mean_squared_error(y_[test_idx], pred_test)
        error_tr += mean_squared_error(y_[train_idx],pred_train)
    print error_tr/nfolds
    print error_ts/nfolds


    

In [258]:
Z=Z_Transform(x,2)
poly_ten_fold_validation_verify(Z,y)

0.258184561507
0.259616633312


In [259]:
'''Function splitting the train data and test data into various splits
    Input 1: 'Z' - the tranformed matrix eg., [1 x x**2 ....]
    Input 2: True predicted values
    Input 3: No of folds (10 by default)
    
    Performance measures such as mean suared error and relative suared error are calculated in this function

'''


def poly_fit_test(x,xp,y):
    train_index = int(0.7*len(y))
    x_train = x[:train_index]
    y_train = y[:train_index]
    x_test = x[train_index:]
    y_test = y[train_index:]
    xptest=xp[train_index:]
    y_=np.array(y)
    theta = poly_theta(x_train,y_train)
    pred_train=predict(theta,x_train)
    pred_test = predict(theta,x_test)
    error_tr = mse_calculator(pred_train,y_train)
    error_ts = mse_calculator(pred_test,y_test)
    print error_tr
    print error_ts
    

In [260]:
Z=Z_Transform(x,2)
poly_fit_test(Z,x,y)

0.260109029077
0.254692854924


In [261]:
'''
Function to solve regression problem using iterative solution:
Input 1: Transforrmed x
Input 2: True y values
Input 3: Learning rate or step size
Input 4: Number of iterations



'''
def gradient_descent(x, y):
    zT = x.transpose()
    n = np.shape(x)[1]
    iterations= 10000
    learning_rate = 0.001
    theta = np.ones(n)
    for i in range(0, iterations):
        predicted = predict_grad(x,theta) #predict with the initial theta
        error = predicted - np.array(y) #calculate the error
        grad = np.dot(zT, error) #find the gradient
        theta = theta - learning_rate*grad # update the theta values
    return theta


In [262]:
g=Z_Transform(x,3)
theta = gradient_descent(g, y)
print 'Solution to the regression problem using iterative solution:'
print thetas

Solution to the regression problem using iterative solution:
[  1.02230218e+00   1.00248539e+00   1.00537442e+00  -1.26097513e-02
  -8.47393334e-03  -6.43213699e-03  -2.31159943e-03  -1.64092663e-02
   5.84897669e-04   3.15715316e-03]


In [58]:
alpha = find_alpha(gram,y)
print (gram.transpose() == gram).all() 
print alpha

True
[ -3.24818239e+15  -1.16702162e+16  -4.68178451e+15 ...,  -6.98182228e+14
  -1.76785926e+15   1.97639305e+15]


In [228]:
'''
Function to calculate the Gaussian Kernel K(X,X')
'''

def Gaussian_kernel(d):
    pred=[]
    sig=0.2
    mul=d*d
    num=[sum(-1.*i) for i in mul]   #rowsums
    deno = 2*np.square(sig)
    sim = np.exp(num/deno)
    return sim
    
    

In [229]:
for i in range(len(y)):
    f=np.array(x)
    t=[x[i] for s in range(len(x))] #x(i)
    xx=f-t  # x-x(i)
    b=Gaussian_kernel(xx)
    pred.append(sum(b*np.array(y))/sum(b))
      

In [266]:
error = mse_calculator(pred,y)
print error

0.248805124864
